<a href="https://colab.research.google.com/github/smgarner229/ESP/blob/master/Project1_GeomtryAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction
The notebooks contained within this folder provide a python solution to the Crawford group programming projects (or at least those I choose to complete).  The plan is to complete these in this python notebook, then translate into either a C++ solution, or more likely a Fortran solution.



#Project \#1
Geometry analysis

## Part 1: Read the geometry
This one is a bit silly in this python notebook, I don't want to go through the hassle of mounting and loading my drive, so I'm just copying the geometry into the notebook.

In [ ]:
full_geom="""7
6  0.000000000000     0.000000000000     0.000000000000
6  0.000000000000     0.000000000000     2.845112131228
8  1.899115961744     0.000000000000     4.139062527233
1 -1.894048308506     0.000000000000     3.747688672216
1  1.942500819960     0.000000000000    -0.701145981971
1 -1.007295466862    -1.669971842687    -0.705916966833
1 -1.007295466862     1.669971842687    -0.705916966833
"""


In [ ]:
copy_geom=full_geom
copy_geom=copy_geom.split()
natoms=int(copy_geom[0])
copy_geom=copy_geom[1:]
atoms=[]
temp_atom=[]
for i in range(natoms):
  atoms.append(copy_geom[i*4:(i+1)*4])
  for j in range(len(atoms[i])):
    atoms[i][j]=float(atoms[i][j])
print("\033[1m{:^3s} {:^7s} {:^7s} {:^7s}\033[0m".format("Z","x","y","z"))
for i in range(natoms):
  print("{:^3.0f} {:^ 6.4f} {:^ 6.4f} {:^ 6.4f}".format(atoms[i][0],atoms[i][1],atoms[i][2],atoms[i][3]))

 Z     x       y       z   
 6   0.0000  0.0000  0.0000
 6   0.0000  0.0000  2.8451
 8   1.8991  0.0000  4.1391
 1  -1.8940  0.0000  3.7477
 1   1.9425  0.0000 -0.7011
 1  -1.0073 -1.6700 -0.7059
 1  -1.0073  1.6700 -0.7059


##Part 2: Calculate Bond Distances
I did this by storing all the values, but perhaps what is the smarter choice is to not store the values, just calculate them on the fly.


In [ ]:

def distance(x1,y1,z1,x2,y2,z2):
  return ((x1-x2)**2+(y1-y2)**2+(z1-z2)**2)**0.5

print("\033[1m{:^3} {:^3} {: ^8}\033[0m".format("i","j","distance"))
distance_array=[]
for i in range(len(atoms)):
  for j in range(i+1,len(atoms)):
    #We calculate the bond length and store it.  It might be smarter to not
    # store the value, rather just calculate on the fly
    calc_distance=distance(atoms[i][1],atoms[i][2],atoms[i][3],atoms[j][1],atoms[j][2],atoms[j][3])
    distance_array.append([i,j,calc_distance])
    print("{:^3} {:^3} {: >8.4f}".format(i,j,calc_distance))
     

 i   j  distance
 0   1    2.8451
 0   2    4.5540
 0   3    4.1991
 0   4    2.0652
 0   5    2.0741
 0   6    2.0741
 1   2    2.2980
 1   3    2.0981
 1   4    4.0434
 1   5    4.0513
 1   6    4.0513
 2   3    3.8133
 2   4    4.8404
 2   5    5.8915
 2   6    5.8915
 3   4    5.8746
 3   5    4.8384
 3   6    4.8384
 4   5    3.3897
 4   6    3.3897
 5   6    3.3399


##Part 3: Bond Angles
The instructions place a filter here, only calculating angles that are reasonable (ie, between close atoms), as some angles aren't all that interesting.  I'm going to calculate all of them, for loops simplicity sake.

In [ ]:
# Helper functions
def get_vector(x1,y1,z1,x2,y2,z2,normalize=True):
  """
  Returns an array with xyz components pointing from 
  atom 1 to atom 2
  """
  if normalize:
    magnitude=distance(x1,y1,z1,x2,y2,z2)
  else:
    magnitude=1.0
  return [(x2-x1)/magnitude,(y2-y1)/magnitude,(z2-z1)/magnitude]

def dot_product(vec1,vec2):
  """
  Returns the dot product of two vectors
  """
  return vec1[0]*vec2[0] + vec1[1]*vec2[1] + vec1[2]*vec2[2]

import math

print("\033[1m{:^3} {:^3} {:^3} {: >8s}\033[0m".format("i","j","k","Angle"))
bond_angles=[]
for i in range(natoms):
  for j in range(i+1,natoms):
    for k in range(j+1,natoms):
      if (i!=j) and (j!=k) and (i!=k):
       if (distance(atoms[i][1],atoms[i][2],atoms[i][3],atoms[j][1],atoms[j][2],atoms[j][3])<4.0) \
         and distance(atoms[k][1],atoms[k][2],atoms[k][3],atoms[j][1],atoms[j][2],atoms[j][3])<4.0: 
          ij_vec=get_vector(atoms[i][1],atoms[i][2],atoms[i][3],atoms[j][1],atoms[j][2],atoms[j][3])
          jk_vec=get_vector(atoms[k][1],atoms[k][2],atoms[k][3],atoms[j][1],atoms[j][2],atoms[j][3])
          angle=math.acos(dot_product(ij_vec,jk_vec))*(180/math.pi)
          print("{:^3} {:^3} {:^3} {: >8.4f}".format(i,j,k,angle))
          bond_angles.append([i,j,k,angle])

 i   j   k     Angle
 0   1   2  124.2683
 0   1   3  115.4793
 0   4   5   35.1095
 0   4   6   35.1095
 0   5   6   36.3737
 1   2   3   28.3774
 4   5   6   60.4845


##Part 4: Out of Plane Angles
We need to build in a cross prodcut here, but that is no worry

In [ ]:
#Helper Functions

def cross_product(vec1,vec2):
  """
  Returns a vector that is the cross product of two vectors
  """
  return [vec1[1]*vec2[2]-vec1[2]*vec2[1],-vec1[0]*vec2[2]+vec1[2]*vec2[0],vec1[0]*vec2[1]-vec1[1]*vec2[0]]

import math

print("\033[1m{:^3s} {:^3s} {:^3s} {:^3s} {: >8s}\033[0m".format("i","j","k","l","Angle"))
out_of_plane_angles=[]
for i in range(natoms):
  for j in range(natoms):
    for k in range(natoms):
      for l in range(j+1,natoms):
        if (i!=j)and (j!=k) and (k!=l) and (i!=k) and (i!=l) and (j!=l):
          if (distance(atoms[j][1],atoms[j][2],atoms[j][3],atoms[k][1],atoms[k][2],atoms[k][3])<4.0) and\
             (distance(atoms[l][1],atoms[l][2],atoms[l][3],atoms[k][1],atoms[k][2],atoms[k][3])<4.0) and \
             (distance(atoms[k][1],atoms[k][2],atoms[k][3],atoms[i][1],atoms[i][2],atoms[i][3])<4.0):
            #Get two vectors to do a cross product
            kj_vec=get_vector(atoms[j][1],atoms[j][2],atoms[j][3],atoms[k][1],atoms[k][2],atoms[k][3])
            kl_vec=get_vector(atoms[l][1],atoms[l][2],atoms[l][3],atoms[k][1],atoms[k][2],atoms[k][3])
            angle=math.sin(math.acos(dot_product(kl_vec,kj_vec)))
            cross_part=cross_product(kj_vec,kl_vec)
            ki_vec=get_vector(atoms[k][1],atoms[k][2],atoms[k][3],atoms[i][1],atoms[i][2],atoms[i][3])
            oopa=math.asin(dot_product(cross_part,ki_vec)/angle)*(180./math.pi)
            print("{:^3d} {:^3d} {:^3d} {:^3d} {: >8.4f}".format(i,j,k,l,oopa))
            out_of_plane_angles.append([i,j,k,l,oopa])


 i   j   k   l     Angle
 0   2   1   3    0.0000
 0   4   5   6   19.8505
 0   4   6   5  -19.8505
 0   5   4   6  -19.9397
 1   4   0   5  -53.6788
 1   4   0   6   53.6788
 1   5   0   6  -54.9771
 2   0   1   3    0.0000
 3   0   1   2    0.0000
 4   0   5   6  -29.8857
 4   0   6   5   29.8857
 4   1   0   5   53.6515
 4   1   0   6  -53.6515
 4   5   0   6   54.8700
 5   0   4   6   30.5590
 5   0   6   4  -31.0643
 5   1   0   4  -53.6263
 5   1   0   6   56.2771
 5   4   0   6  -56.1946
 6   0   4   5  -30.5590
 6   0   5   4   31.0643
 6   1   0   4   53.6263
 6   1   0   5  -56.2771
 6   4   0   5   56.1946


##Part 5: Torsional Angles


In [ ]:
import math
print("\033[1m{:^3s} {:^3s} {:^3s} {:^3s} {: >8s}\033[0m".format("i","j","k","l","Angle"))

torsion_dihedral=[]
for i in range(natoms):
  for j in range(i+1,natoms):
    for k in range(j+1,natoms):
      for l in range(k+1,natoms):
        if (i!=j)and (j!=k) and (k!=l) and (i!=k) and (i!=l) and (j!=l):
          if (distance(atoms[i][1],atoms[i][2],atoms[i][3],atoms[j][1],atoms[j][2],atoms[j][3])<4.0) and\
             (distance(atoms[j][1],atoms[j][2],atoms[j][3],atoms[k][1],atoms[k][2],atoms[k][3])<4.0) and \
             (distance(atoms[k][1],atoms[k][2],atoms[k][3],atoms[l][1],atoms[l][2],atoms[l][3])<4.0):
             ij_vec=get_vector(atoms[i][1],atoms[i][2],atoms[i][3],atoms[j][1],atoms[j][2],atoms[j][3])
             jk_vec=get_vector(atoms[k][1],atoms[k][2],atoms[k][3],atoms[j][1],atoms[j][2],atoms[j][3])
             kj_vec=get_vector(atoms[j][1],atoms[j][2],atoms[j][3],atoms[k][1],atoms[k][2],atoms[k][3])
             kl_vec=get_vector(atoms[l][1],atoms[l][2],atoms[l][3],atoms[k][1],atoms[k][2],atoms[k][3])
             term1=cross_product(ij_vec,jk_vec)
             term2=cross_product(kj_vec,kl_vec)
             bottom1=math.sin(math.acos(dot_product(ij_vec,jk_vec)))
             bottom2=math.sin(math.acos(dot_product(kj_vec,kl_vec)))
             acosterm=dot_product(term1,term2)/(bottom1*bottom2)
             if (acosterm>1.0):
               acosterm=1.0
             elif (acosterm<-1.0):
               acosterm=-1.0
             tau=math.acos(acosterm)*(180/math.pi)
             print("{:^3d} {:^3d} {:^3d} {:^3d} {: >8.4f}".format(i,j,k,l,tau))

 i   j   k   l     Angle
 0   1   2   3  180.0000
 0   4   5   6   36.3668


##Part \#5: Center of Mass
As of right now I'm hacking in the masses, but then again the whole thing is more or less a hack job right now.  Eventually it should be cleaned up and turned into proper data storage.

Be careful here, we're going to update the coordinates to the COM frame, so if you want to run this twice, you'll need to re-establish the original coordinates.


In [ ]:
z_to_m={'1':1.,'6':12.,'7':14.,'8':16.}
xcom,ycom,zcom=0.0,0.0,0.0
totalmass=0.0
for i in range(natoms):
  mass=z_to_m[str(int(atoms[i][0]))]
  totalmass+=mass
  xcom+=mass*atoms[i][1]
  ycom+=mass*atoms[i][2]
  zcom+=mass*atoms[i][3]

xcom/=totalmass
ycom/=totalmass
zcom/=totalmass

print(xcom,ycom,zcom)

import copy
com_geom=copy.deepcopy(atoms)

for i in range(natoms):
  com_geom[i][1]-=xcom
  com_geom[i][2]-=ycom
  com_geom[i][3]-=zcom



0.6459026583098636 0.0 2.3182057901600683


##Part \#6: Moments of Inertia
These will be straight forward using the COM coords.  In the principal coordinate frame, the inertial tensor is diagonal, so calculating it this way is preferred.  

Working not in the principal axis frame, this is not true, and we'll do it this way as well for practice diagonalizing a small matrix.

I was being silly above.  For highly symmetric molecules, this is true, but this molecule is not symmetric enough.  We need to be in the principal axis system to get the internal moments of inertia as opposed to rotating about some other coordinate system.  

In [ ]:
Ixx,Iyy,Izz=0.0,0.0,0.0
Ixy,Iyz,Ixz=0.0,0.0,0.0
for i in range(natoms):
  mass=z_to_m[str(int(atoms[i][0]))]
  
  Ixx+=mass*(com_geom[i][2]**2+com_geom[i][3]**2)
  Iyy+=mass*(com_geom[i][1]**2+com_geom[i][3]**2)
  Izz+=mass*(com_geom[i][1]**2+com_geom[i][2]**2)

  Ixy-=mass*(com_geom[i][1]*com_geom[i][2])
  Iyz-=mass*(com_geom[i][2]*com_geom[i][3])
  Ixz-=mass*(com_geom[i][1]*com_geom[i][3])

import numpy as np
inertmat=np.diag((Ixx,Iyy,Izz))
inertmat[1][0]=Ixy
inertmat[0][1]=Ixy
inertmat[0][2]=Ixz
inertmat[2][0]=Ixz
inertmat[1][2]=Iyz
inertmat[2][1]=Iyz

from numpy import linalg as LA
mat,eigvecs=LA.eig(inertmat)
mat.sort()
print("\033[1m{:^6s} {:^6s} {:^6s}\033[0m".format("Ixx","Iyy","Izz"))
print("{:^6.2f} {:^6.2f} {:^6.2f}\tamu*bohr*bohr".format(mat[0],mat[1],mat[2]))
print("{:^6.2f} {:^6.2f} {:^6.2f}\tamu*AA*AA".format(mat[0]*0.529177*0.529177,mat[1]*0.529177*0.529177,mat[2]*0.529177*0.529177))


 Ixx    Iyy    Izz  
31.81  178.40 199.06	amu*bohr*bohr
 8.91  49.96  55.74 	amu*AA*AA


##Part \#8: 

In [ ]:
import math

A=(6.626e-34*6.022e+26)/(8*math.pi*math.pi*(3*10**10)*mat[0]*(10**-10)*(10**-10))*1.88973*1.88973
B=(6.626e-34*6.022e+26)/(8*math.pi*math.pi*(3*10**10)*mat[1]*(10**-10)*(10**-10))*1.88973*1.88973
C=(6.626e-34*6.022e+26)/(8*math.pi*math.pi*(3*10**10)*mat[2]*(10**-10)*(10**-10))*1.88973*1.88973
print("\033[1m{:^6s} {:^6s} {:^6s}\033[0m".format("A","B","C"))
print("{:^6.3f} {:^6.3f} {:^6.3f}\tcm-1".format(A,B,C))

  A      B      C   
1.891  0.337  0.302 	cm-1
